# Chapter 6 - Prompt Engineering

In [1]:
%%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [2]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"


## 加载模型

In [3]:
# 导入依赖
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",  
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    return_full_text=False,
    do_sample=False
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'do_sample', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [4]:
# 创建prompt
message = [
    {"role":"user", "content":"请编写一个跟猫有关的笑话"}
]

# 生成输出
output = pipe(message)
print(output[0]['generated_text'])

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


当然可以，以下是一个关于猫的有趣笑话：

为什么猫咪总是喜欢在窗台上晒太阳？
因为它们知道，这是它们最喜欢的“日光浴”！


In [6]:
# 应用提示词对话模
prompt = pipe.tokenizer.apply_chat_template(message,tokenize=False)
print(prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
请编写一个跟猫有关的笑话<|im_end|>



In [7]:
# 使用一个高的temperature。温度越高，随机性越强
output = pipe(message, do_sample = True, temperature = 1)
print(output[0]["generated_text"])

Passing `generation_config` together with generation-related arguments=({'do_sample', 'temperature'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


当然，以下是一个与猫相关的有趣笑话：

在一只名叫“毛毛”的猫咪眼中，有一只叫“小灰”的老鼠。一天，毛毛和小灰一起玩捉迷藏游戏，毛毛不小心把小灰藏起来了。小灰非常生气，说：“你这个笨蛋！我才是藏起来的，你怎么能让我来找呢？”

毛毛很恼火，开始用爪子抓着小灰。小灰看着毛毛的样子，突然想起他平时喜欢听的故事《小熊理查德》里的故事。于是小灰说道：“哦，原来是这样啊。不过现在我可没有时间听你的故事了，我要去找吃的了。”

毛毛被这只可爱的“小灰”吓得不敢靠近，只好乖乖地留在原地，等待着它的食物。

这就是“毛毛与小灰”这一对小家伙之间的一个小小的幽默故事。希望这个笑话能够给各位带来一些欢乐！


In [9]:
# 使用一个高的top_p
output = pipe(message, do_sample = True, top_p = 1)
print(output[0]["generated_text"])

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


在一片宁静的草地上，一只小猫咪正在悠闲地吃着新鲜的胡萝卜。突然，它看到了远处有只活泼的小狗正在追逐一只飞过的蝴蝶。小狗停下脚步，惊讶地看着那只小猫，问：“你怎么会在这里？”猫咪答道：“我只是享受我的独处时间。”

小狗听了，开始用尖锐的声音叫喊：“别吃我！别吃我！”猫咪微笑着回答：“你看起来很有趣，但我更喜欢安静地吃胡萝卜。”

小狗继续奔跑，而小猫则在一旁静静地看着，似乎也在享受这个短暂的平静时光。

就这样，两人成为了彼此生活中的朋友，分享着一天的快乐和烦恼。这个故事告诉我们，有时候，简单的生活就是最好的调剂。


# 高级提示词工程
## 复杂提示

In [12]:
text = """在上一篇文章中，我们探讨了注意力机制——现代深度学习模型中普遍采用的一种方法。注意力机制的概念有助于提升神经机器翻译应用的性能。在本文中，我们将介绍 Transformer 模型——一种利用注意力机制来提升模型训练速度的模型。Transformer 模型在某些特定任务上的表现优于谷歌神经机器翻译模型。然而，其最大的优势在于 Transformer 模型本身就非常适合并行化。事实上，谷歌云建议使用 Transformer 模型作为参考模型来使用其 Cloud TPU 服务。接下来，我们将深入剖析该模型，探究其工作原理。

Transformer 模型最初在论文《Attention is All You Need》中提出。TensorFlow 实现了该模型，并将其包含在 Tensor2Tensor 包中。哈佛大学的自然语言处理小组编写了一份指南，用 PyTorch 实现标注了这篇论文。在这篇文章中，我们将尝试对概念进行一些简化，并逐一介绍，希望能帮助那些对该主题没有深入了解的人更容易理解。

首先，让我们把模型看作一个黑盒。在机器翻译应用中，它会接收一个句子，并输出另一种语言的翻译。

打开这个“黑盒”，我们可以看到编码组件、解码组件以及它们之间的连接。

编码组件是一叠编码器（论文中将六个编码器堆叠在一起——数字六并没有什么特别之处，当然可以尝试其他排列方式）。解码组件是一叠相同数量的解码器。

所有编码器的结构都相同（但它们的权重并不共享）。每个编码器又分为两个子层：

编码器的输入首先流经一个自注意力层——该层帮助编码器在编码特定单词时关注输入句子中的其他单词。我们将在本文后面更详细地探讨自注意力机制。

自注意力层的输出被送入一个前馈神经网络。同一个前馈网络被独立地应用于每个位置。

解码器包含这两个层，但在它们之间有一个注意力层，它帮助解码器专注于输入句子的相关部分（类似于注意力在序列到序列模型中的作用）。

现在我们已经了解了模型的主要组成部分，接下来让我们看看各种向量/张量以及它们如何在这些组件之间流动，从而将训练好的模型的输入转换为输出。

与一般的自然语言处理应用一样，我们首先使用嵌入算法将每个输入词转换为一个向量。

每个词都被嵌入到一个大小为 512 的向量中。我们将用这些简单的方框来表示这些向量。

嵌入操作只发生在最底层的编码器中。所有编码器的共同抽象是，它们接收一个大小为 512 的向量列表——在最底层的编码器中，它是词嵌入；而在其他编码器中，它是其下方编码器的输出。该列表的大小是一个我们可以设置的超参数——它基本上等于训练数据集中最长句子的长度。

在对输入序列中的单词进行嵌入后，每个单词都会依次经过编码器的两个子层。

在这里，我们开始看到 Transformer 的一个关键特性：每个位置的单词在编码器中都有其自身的路径。自注意力层中的这些路径之间存在依赖关系。然而，前馈层不存在这些依赖关系，因此各个路径可以在经过前馈层时并行执行。

接下来，我们将示例切换到一个更短的句子，并观察编码器每个子层中发生的情况。

现在开始编码！

正如我们之前提到的，编码器接收一个向量列表作为输入。它通过将这些向量传递给“自注意力”层来处理该列表，然后再传递给前馈神经网络，最后将输出发送到下一个编码器。
"""

# 提示词部分
persona = "你是大型语言模型的专家。你擅长将复杂的论文分解成易于理解的摘要。\n"
instruction = "总结所提供论文的关键发现.\n"
context = "你的总结应提取最关键的信息点，以帮助研究人员快速了解论文中最重要内容.\n"
data_format = "创建一个要点总结，概述方法。随后用一个简洁的段落概括主要结果。\n"
audience = "该摘要专为忙碌的研究人员设计，他们需要快速掌握大型语言模型的新趋势\n"
tone = "语气应专业且清晰。\n"

data = f"需要总结的文本:{text}"

query = persona + instruction + context + data_format + audience + tone + data

In [13]:
messages = [
    {"role":"user","content":query}
]

print(tokenizer.apply_chat_template(messages, tokenize = False))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
你是大型语言模型的专家。你擅长将复杂的论文分解成易于理解的摘要。
总结所提供论文的关键发现.
你的总结应提取最关键的信息点，以帮助研究人员快速了解论文中最重要内容.
创建一个要点总结，概述方法。随后用一个简洁的段落概括主要结果。
该摘要专为忙碌的研究人员设计，他们需要快速掌握大型语言模型的新趋势
语气应专业且清晰。
需要总结的文本:在上一篇文章中，我们探讨了注意力机制——现代深度学习模型中普遍采用的一种方法。注意力机制的概念有助于提升神经机器翻译应用的性能。在本文中，我们将介绍 Transformer 模型——一种利用注意力机制来提升模型训练速度的模型。Transformer 模型在某些特定任务上的表现优于谷歌神经机器翻译模型。然而，其最大的优势在于 Transformer 模型本身就非常适合并行化。事实上，谷歌云建议使用 Transformer 模型作为参考模型来使用其 Cloud TPU 服务。接下来，我们将深入剖析该模型，探究其工作原理。

Transformer 模型最初在论文《Attention is All You Need》中提出。TensorFlow 实现了该模型，并将其包含在 Tensor2Tensor 包中。哈佛大学的自然语言处理小组编写了一份指南，用 PyTorch 实现标注了这篇论文。在这篇文章中，我们将尝试对概念进行一些简化，并逐一介绍，希望能帮助那些对该主题没有深入了解的人更容易理解。

首先，让我们把模型看作一个黑盒。在机器翻译应用中，它会接收一个句子，并输出另一种语言的翻译。

打开这个“黑盒”，我们可以看到编码组件、解码组件以及它们之间的连接。

编码组件是一叠编码器（论文中将六个编码器堆叠在一起——数字六并没有什么特别之处，当然可以尝试其他排列方式）。解码组件是一叠相同数量的解码器。

所有编码器的结构都相同（但它们的权重并不共享）。每个编码器又分为两个子层：

编码器的输入首先流经一个自注意力层——该层帮助编码器在编码特定单词时关注输入句子中的其他单词。我们将在本文后面更详细地探讨自注意力机制。

自注

In [15]:
# 生成输出
outputs = pipe(messages)
print(outputs[0]["generated_text"])

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


在上一篇文章中，我们讨论了注意力机制在现代深度学习模型中的广泛应用，特别是如何提升神经机器翻译的应用性能。本文进一步介绍了Transformer模型，这是一种利用注意力机制来提高模型训练效率的模型。Transformer模型在某些特定任务上表现出色，如谷歌神经机器翻译模型。然而，它的最大优势在于其高度可扩展性，这得益于其并行化的特性。此外，Google Cloud也推荐使用Transformer模型作为参考模型，以便更好地利用Cloud TPU服务。接下来，我们将深入分析Transformer模型的工作原理，包括其基本组成和工作流程。


## 情景学习


In [20]:
# 使用一个虚构单词的句子示例
one_shot_prompt = [
    {
        "role": "user",
        "content": "'嘻嘻哈哈'是一种传统的乐器，一个使用嘻嘻哈哈的句子是："
    },
    {
        "role": "assistant",
        "content": "我有一把嘻嘻哈哈，这是我叔叔送我的生日礼物，我喜欢在家里弹奏它。"
    },
    {
        "role": "user",
        "content": "对某物进行'嘿哈'是指挥剑攻击它，一个使用嘿哈的句子是："
    }
]
print()

In [21]:
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
'嘻嘻哈哈'是一种传统的乐器，一个使用嘻嘻哈哈的句子是：<|im_end|>
<|im_start|>assistant
我有一把嘻嘻哈哈，这是我叔叔送我的生日礼物，我喜欢在家里弹奏它。<|im_end|>
<|im_start|>user
对某物进行'嘿哈'是指挥剑攻击它，一个使用嘿哈的句子是：<|im_end|>



In [22]:
# 获取输出
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


嘿哈！这把剑真厉害，每次挥动都能打出好几下。


## 链式提示：将问题分解

In [25]:
# 为产品创建名称和口号
product_prompt = [
    {
        "role":"user",
        "content":"为利用大型语言模型的聊天机器人创建一个名称和口号"
    }
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


为了创建一个吸引人的名称和口号，我将考虑以下几个因素：

1. **简洁性**：一个好的名字应该简单易记，能够传达核心信息。
2. **独特性**：选择一个与众不同的名称可以增加品牌识别度。
3. **创新性**：考虑到人工智能技术的发展趋势，创造一些新颖且具有吸引力的名字。
4. **情感共鸣**：口号应能引起用户的共鸣，激发他们对特定主题的兴趣。

基于以上考虑，这里有一个名为“智言通语”的名称，以及一个口号：“智言通语，智能对话”。这个名称既体现了AI技术的特点，又突出了其与用户沟通的方式。口号则通过强调“智言”（智慧的话语）和“通语”（交流的语言），表达了机器与人类之间的桥梁作用，同时也传递了智能化、便捷化的理念。

这样的命名和口号不仅符合当前AI技术发展的潮流，还能够有效地吸引用户使用你的产品或服务。


In [ ]:
# 根据产品的名称和口号，生成一则销售口号
sales_prompt = [
    {
        "role":"user", 
        "content":f"为以下产品生成一个非常简短的推销口号：{product_description}"
    }  
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"智言通语，智能对话，让沟通更智能！"


# 使用生成模型进行推理
## 思维链：回答前先思考

In [ ]:
# 不作明确推理回答


## 当前章节的内容省略


# 提示词设计原则
1. Prompt generator：提示词不是一次写好的。可以先让大模型生成一版提示词，然后再去逐步迭代优化
2. Be clear and dircet：提示词越清晰越好
3. Use examples(multishot)：给模型更多的参考例子,例子和要求要统一
4. Let Model think (chain for thought)：给模型思考的时间
   1. think it step by step
   2. <think>xxx 1.step 2.step</think> 明确每一步思考的内容
5. Use XML tags：使用xml标签，尤其是大段的、 容易让大模型混乱的带有缩进之类的东西。标签可以提供清晰的边界
6. Give Model a role (system prompt)：给模型一个具体的角色。电力工程师、数学老师...
7. Prefill Model`s response：让模型在给定的内容之后接着生成（使用generate的API而不是response）
8. Chain complex prompt：workflow，提示词和任务进行拆分，一类任务对应一个提示词
9. Long context tips：长上下文的内容要放在instruction或prompt前面的部分。模型会优先遵循最近最近的一个指令